In [26]:
# print the column names of the arxiv_papers.db
import sqlite3

conn = sqlite3.connect('arxiv_papers.db')
cur = conn.cursor()
cur.execute("PRAGMA table_list")
print(cur.fetchall())

[('main', 'arxiv_to_s2', 'table', 2, 0, 0), ('main', 'citations', 'table', 2, 0, 0), ('main', 'papers', 'table', 6, 0, 0), ('main', 'arxiv_papers', 'table', 6, 0, 0), ('main', 'sqlite_schema', 'table', 5, 0, 0), ('temp', 'sqlite_temp_schema', 'table', 5, 0, 0)]
